# Dynamic Resource Allocation with MLOPS integration in Telecommunication Cloud Environments 

### Life Cycle of the Project

* Understanding the Problem Statement
* Data Collection
* Data-preprocessing
* Exploratory Data Analysis
* Model Training
* Model Selection

### 1. Problem Statement

Telecommunication cloud systems need to manage and allocate resources like computing power, storage, and bandwidth to handle changing user demands. The challenge is to do this efficiently while ensuring good service quality, reducing costs, and saving energy. These systems also need to handle multiple virtual networks, make quick decisions, and stay reliable even if something goes wrong. The goal is to create a system that uses resources wisely, adapts to changes, and works smoothly for all users.

### 2. Data Collection

* Data Source : https://data.niaid.nih.gov/resources?id=zenodo_10245447


* Data consists in three csv files - 


        * pods_request_workloads.csv - It consists 7 Columns and Rows
                                (which are 'timestamp', 'uid', 'node', 'cpu', 'memory', 'nvidia_com', 'scenario')

        * nodes_allocatable.csv - It consists 8 Columns and Rows
                                (which are 'timestamp', 'node', 'cpu', 'memory', 'nvidia_com', 'status', 'condition', 'scenario')


We'll merge the dataset because most of the attributes are same 

##### 2.1 Import Data and Required Packages  / Libraries

* IMPORTING LIBRARIES

In [1]:
import numpy as numpy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder # type: ignore
from sklearn.preprocessing import StandardScaler

* IMPORTING DATASETS

In [2]:
df1 = pd.read_csv('data/nodes_allocatable.csv')
df2 = pd.read_csv('data/pods_request_workloads.csv')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17412\2202315815.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('data/nodes_allocatable.csv')


In [3]:
# Merge datasets with an outer join
data = pd.merge(df1,df2,
    on=['timestamp', 'node'],  # Merge on common columns
    how='outer',  # Include all rows from both datasets
    suffixes=('_workloads', '_allocatable')  # Add suffixes to avoid column name conflicts
)

# Display the merged dataset
print("Merged Dataset Shape:", data.shape)
print("Merged Dataset Columns:", data.columns)

# Save the merged dataset for further use
data.to_csv('data.csv', index=False)

Merged Dataset Shape: (22237197, 13)
Merged Dataset Columns: Index(['timestamp', 'node', 'cpu_workloads', 'memory_workloads',
       'nvidia_com_gpu_workloads', 'status', 'condition', 'scenario_workloads',
       'uid', 'cpu_allocatable', 'memory_allocatable',
       'nvidia_com_gpu_allocatable', 'scenario_allocatable'],
      dtype='object')


##### 2.2 Dataset Information

* TOP 5 RECORDS OF THE DATASET

In [4]:
data.head()

,timestamp,node,cpu_workloads,memory_workloads,nvidia_com_gpu_workloads,status,condition,scenario_workloads,uid,cpu_allocatable,memory_allocatable,nvidia_com_gpu_allocatable,scenario_allocatable
0,2023-10-13 12:04:00,0xozF0md0I,7.91,14684.878906,0.0,NaN,NaN,1,0a7e3149-7520-44af-be02-6cb0ede2109d,0.100,0.0,0.0,A
1,2023-10-13 12:04:00,0xozF0md0I,7.91,14684.878906,0.0,NaN,NaN,1,327fff0d-3d9d-4c99-8e0d-f8581dfa7373,0.010,40.0,0.0,A
2,2023-10-13 12:04:00,0xozF0md0I,7.91,14684.878906,0.0,NaN,NaN,1,40054bfd-b720-4f90-86c2-974e227c178f,0.025,0.0,0.0,A
3,2023-10-13 12:04:00,1uYdt27oKb,7.91,14684.878906,0.0,NaN,NaN,1,234bebd4-85d4-4068-8dde-0b2a43fc7940,0.100,0.0,0.0,A
4,2023-10-13 12:04:00,1uYdt27oKb,7.91,14684.878906,0.0,NaN,NaN,1,8619d1f4-7029-4425-9f99-725dcb545709,0.010,40.0,0.0,A


* TOTAL NO OF ROWS & COLUMNS PRESENT IN THE DATASET

In [5]:
rows,cols=data.shape
print(f"Total no of Rows in df1 \t\t:\t{rows} \nTotal no of Columns in dataset \t:\t{cols}")

Total no of Rows in df1 		:	22237197 
Total no of Columns in dataset 	:	13


* DATATYPES OF THE ATTRIBUTES PRESENT IN THE DATASET 

In [6]:
data.dtypes

timestamp                      object
node                           object
cpu_workloads                 float64
memory_workloads              float64
nvidia_com_gpu_workloads      float64
status                         object
condition                      object
scenario_workloads              int64
uid                            object
cpu_allocatable               float64
memory_allocatable            float64
nvidia_com_gpu_allocatable    float64
scenario_allocatable           object
dtype: object

* DESCRIPTION OF THE DATASET

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22237197 entries, 0 to 22237196
Data columns (total 13 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   timestamp                   object 
 1   node                        object 
 2   cpu_workloads               float64
 3   memory_workloads            float64
 4   nvidia_com_gpu_workloads    float64
 5   status                      object 
 6   condition                   object 
 7   scenario_workloads          int64  
 8   uid                         object 
 9   cpu_allocatable             float64
 10  memory_allocatable          float64
 11  nvidia_com_gpu_allocatable  float64
 12  scenario_allocatable        object 
dtypes: float64(6), int64(1), object(6)
memory usage: 2.2+ GB


### 3. Data Preprocessing

 3.1 Data Checks to Perform

*  Check Missing Values

In [8]:
data.isnull().sum()

timestamp                           0
node                                0
cpu_workloads                       0
memory_workloads                    0
nvidia_com_gpu_workloads            0
status                        1306071
condition                     1306071
scenario_workloads                  0
uid                                 0
cpu_allocatable                     0
memory_allocatable                  0
nvidia_com_gpu_allocatable          0
scenario_allocatable                0
dtype: int64

*  Check Duplicates

In [9]:
data.duplicated().sum()

1116

* Feature Engineering

In [10]:
# Workload-to-Allocatable Ratios
data['cpu_ratio'] = data['cpu_workloads'] / data['cpu_allocatable']

In [11]:
# Total Workload
data['total_workload'] = data['cpu_workloads'] + data['memory_workloads'] + data['nvidia_com_gpu_workloads']

In [12]:
data['cpu_memory_interaction'] = data['cpu_workloads'] * data['memory_workloads']
data['workload_allocatable_interaction'] = (data['cpu_workloads'] * data['cpu_allocatable']) +  (data['memory_workloads'] * data['memory_allocatable'])

* Converting the Object into Int

In [13]:
data.dtypes

timestamp                            object
node                                 object
cpu_workloads                       float64
memory_workloads                    float64
nvidia_com_gpu_workloads            float64
status                               object
condition                            object
scenario_workloads                    int64
uid                                  object
cpu_allocatable                     float64
memory_allocatable                  float64
nvidia_com_gpu_allocatable          float64
scenario_allocatable                 object
cpu_ratio                           float64
total_workload                      float64
cpu_memory_interaction              float64
workload_allocatable_interaction    float64
dtype: object

In [14]:
data['timestamp']=data['timestamp'].str.replace(r'[^\d]','',regex='True')
data['timestamp']=pd.to_numeric(data['timestamp'], errors='coerce').astype('int64')

In [15]:
data['cpu_workloads']=pd.to_numeric(data['cpu_workloads'], errors='coerce').astype('int64')
data['memory_workloads']=pd.to_numeric(data['memory_workloads'], errors='coerce').astype('int64')
data['nvidia_com_gpu_workloads']=pd.to_numeric(data['nvidia_com_gpu_workloads'], errors='coerce').astype('int64')
data['cpu_allocatable']=pd.to_numeric(data['cpu_allocatable'], errors='coerce').astype('int64')
data['memory_allocatable']=pd.to_numeric(data['memory_allocatable'], errors='coerce').astype('int64')
data['nvidia_com_gpu_allocatable']=pd.to_numeric(data['nvidia_com_gpu_allocatable'], errors='coerce').astype('int64')
data['cpu_ratio']=pd.to_numeric(data['cpu_ratio'], errors='coerce').astype('int64')
data['total_workload']=pd.to_numeric(data['total_workload'], errors='coerce').astype('int64')
data['cpu_memory_interaction']=pd.to_numeric(data['cpu_memory_interaction'], errors='coerce').astype('int64')
data['workload_allocatable_interaction']=pd.to_numeric(data['workload_allocatable_interaction'], errors='coerce').astype('int64')

In [16]:
label_encoder = LabelEncoder()

categorical = ['node', 'status', 'condition', 'scenario_workloads', 'scenario_allocatable', 'uid']

for col in categorical:
    data[col] = label_encoder.fit_transform(data[col])

In [17]:
data.head()

,timestamp,node,cpu_workloads,memory_workloads,nvidia_com_gpu_workloads,status,condition,scenario_workloads,uid,cpu_allocatable,memory_allocatable,nvidia_com_gpu_allocatable,scenario_allocatable,cpu_ratio,total_workload,cpu_memory_interaction,workload_allocatable_interaction
0,20231013120400,0,7,14684,0,1,2,0,429,0,0,0,0,79,14692,116157,0
1,20231013120400,0,7,14684,0,1,2,0,2044,0,40,0,0,791,14692,116157,587395
2,20231013120400,0,7,14684,0,1,2,0,2559,0,0,0,0,316,14692,116157,0
3,20231013120400,1,7,14684,0,1,2,0,1435,0,0,0,0,79,14692,116157,0
4,20231013120400,1,7,14684,0,1,2,0,5378,0,40,0,0,791,14692,116157,587395


In [18]:
scaler = StandardScaler()
data= scaler.fit_transform(data)

In [19]:
data

array([[-0.77377799, -1.89789768, -1.47976616, ..., -1.43977843,
        -2.10074719, -0.54800191],
       [-0.77377799, -1.89789768, -1.47976616, ..., -1.43977843,
        -2.10074719, -0.54394284],
       [-0.77377799, -1.89789768, -1.47976616, ..., -1.43977843,
        -2.10074719, -0.54800191],
       ...,
       [ 3.27908954,  1.79608399, -0.70162823, ...,  1.47786161,
         0.52475876,  0.33185503],
       [ 3.27908954,  1.79608399, -0.70162823, ...,  1.47786161,
         0.52475876,  0.33185503],
       [ 3.27908954,  1.79608399, -0.70162823, ...,  1.47786161,
         0.52475876,  0.33185503]])